In [2]:
import scipy.sparse as spr
import scipy.sparse.linalg as spla
import sys
path2oti = '../../../build/'
sys.path.append(path2oti) # Add path to OTI library.

import pyoti.real   as r
import pyoti.sparse as oti 
import pyoti.core   as coti
import pyoti.fem    as fem 

import pyoti.static.onumm1n1   as dual
import pyoti.static.mdnum2     as md2
import pyoti.static.onumm1n10  as om1n10
import pyoti.static.onumm1n2   as om1n2

%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

e  = oti.e
np = oti.np

import pyvista as pv
# p = pv.BackgroundPlotter()
# pv.set_plot_theme("document")


In [13]:
fem.set_global_algebra(om1n10)
alg = fem.get_global_algebra()

In [14]:
Th = fem.square(1,1,he=0.003,quads=True,structured=True)
# Th = fem.square(1,1,he=0.1,quads=False,structured=True)
Th.x += 0.5
Th.y += 0.5

Th

< mesh (pyoti.static.onumm1n10) object with 112225 nodes, 112896 elements of types ( point1 (4), line2 (1336), quad4 (111556) ) >

In [67]:
# Perform perturbation of all nodes
order=1
for i in range(Th.x.shape[0]):
    xi = Th.x[i,0]
    yi = Th.y[i,0]
    Th.x[i,0] = xi + 2*(xi.real-0.5) * alg.e(1, order=order)
    Th.y[i,0] = yi + 2*(yi.real-0.5) * alg.e(1, order=order)
# end for 

In [14]:
# Perform perturbation of at least half of the nodes
order=1
for i in range(Th.x.shape[0]):
    xi = Th.x[i,0]
    yi = Th.y[i,0]
    if xi.real >= 0.5:
        Th.x[i,0] = xi + 2*(xi.real-0.5) * alg.e(1, order=order)
        Th.y[i,0] = yi + 2*(yi.real-0.5) * alg.e(2, order=order)
    # end if 

AttributeError: module 'pyoti.real' has no attribute 'e'

In [19]:

grid = Th.to_pv()

p.clear()
p.enable_parallel_projection()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=16,cmap='jet')

(vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7fd14c870be8

In [3]:
#*****************************************************************************************************
def solve_2d_problem_prealloc(Th, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    ndim_analysis = 2
    els = Th.elements[2]

    fem.end_elements()
    
    K = alg.lil_matrix((Th.x.shape[0],Th.x.shape[0]))
    f = alg.zeros((Th.x.shape[0],1))
    funct = 2*(Th.x**2+Th.y**2-Th.x-Th.y)
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)#, nbases=1 order=order)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            NN_tmp1 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            NN_tmp2 = alg.zeros( ( elem.nbasis, elem.nbasis ), nip = elem.nip )
            Kel = alg.zeros( ( elem.nbasis, elem.nbasis ) )
            NT_tmp1 = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            NT_tmp2 = alg.zeros( ( elem.nbasis, 1 ), nip = elem.nip )
            fel = alg.zeros( ( elem.nbasis, 1 ) )
            tmp1 = alg.zero( nip=elem.nip )
        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            # reinitialize values.
#             NN_tmp1.set(0.0)
#             NN_tmp2.set(0.0)
#             NT_tmp1.set(0.0)
            
            elem.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            for ii in range(elem.nbasis):
                fh[ii,0] = funct[ int(elm_nodes[i,ii]),0]
            # end for 
            
            alg.transpose(Nx,out=NT_tmp1)
            alg.transpose(Ny,out=NT_tmp2)
            
            alg.dot(NT_tmp1, Nx, out = NN_tmp1)
            alg.dot(NT_tmp2, Ny, out = NN_tmp2)
            
            alg.neg(NN_tmp1, out = NN_tmp1)
            alg.sub(NN_tmp1,NN_tmp2,out=NN_tmp1)
            
#             tmp = - oti.dot(oti.transpose(Nx),Nx) - oti.dot(oti.transpose(Ny),Ny) 
            
            alg.gauss_integrate(NN_tmp1,elem.dV, out = Kel)
            
            alg.dot_product( fh, N, out = tmp1 )
            alg.transpose(N, out = NT_tmp1)
            alg.mul(tmp1,NT_tmp1, out = NT_tmp2)
            alg.gauss_integrate(NT_tmp2, elem.dV, out = fel)
          
            assemble_globals(Kel,fel,K,f,elm_nodes[i,:])
 
        # end for

    # end for

    fem.end_elements()   
    
    end_assmbly_time = time()
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.elements[1]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_)
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for

    end_bc_time = time()
    
    u = alg.solve(K.tocsr(),f)
#     u = alg.solve_sparse_tests(K.tocsr(),f)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append( end_bc_time      - end_assmbly_time  )
        times['solve'].append(  end_solve_time   - end_bc_time      )
        times['total'].append( end_solve_time   - start_time        )
        
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 
    
    return u,K,f

#-----------------------------------------------------------------------------------------------------

In [4]:
#*****************************************************************************************************
def solve_2d_problem(Th, stats=True):
    global times
    from timeit import default_timer as time
    
    start_time = time()
    
    ndim_analysis = 2
    els = Th.elements[2]
    
#         Th.x.get_active
    
    fem.end_elements()
    
    K = alg.lil_matrix((Th.x.shape[0],Th.x.shape[0]))
    f = alg.zeros((Th.x.shape[0],1))
    funct = 2*(Th.x**2+Th.y**2-Th.x-Th.y)
    # 
    for j in range(els['types'].size):

        elem = fem.element[ els['types'][j] ]

        if not elem.is_allocated():

            elem.end()
            elem.allocate(intorder=2)#,nbases=1, order=order)
            elem.allocate_spatial(ndim_analysis,compute_Jinv = True)

            fh = alg.zeros( ( elem.nbasis, 1 ) )
            
        # end if 

        elm_nodes = els['indices'][j]

        for i in range(elm_nodes.shape[0]):
            
            # reinitialize values.

            elem.set_coordinates(Th.x,Th.y,Th.z,elm_nodes[i,:])
            elem.compute_jacobian()

            Nx = elem.Nx
            Ny = elem.Ny
            N  = elem.N
            
            for ii in range(elem.nbasis):
                fh[ii,0] = funct[ int(elm_nodes[i,ii]),0]
            # end for 
            
            tmp = - alg.dot(alg.transpose(Nx),Nx) - alg.dot(alg.transpose(Ny),Ny) 
            
            Kel = alg.gauss_integrate( tmp, elem.dV )
            
            tmp = alg.dot_product( fh, N)
            fel = alg.gauss_integrate( tmp*alg.transpose(N), elem.dV )
          
            assemble_globals(Kel,fel,K,f,elm_nodes[i,:])
 
        # end for

    # end for

    fem.end_elements()   
    
    end_assmbly_time = time()    
    
    # Setting Dirichlet BCs using TGV.
    TGV = 1e30
    
    # Every 1D node has dirichlet bc = 0
    els = Th.elements[1]
    for j in range(els['types'].size):

        elm_nodes = np.unique(els['indices'][j])

        for ii_ in elm_nodes:
            ii = int(ii_)
            K[ii,ii] = TGV
            # end for
            f[ii,0] = 0.0
        # end for 
        
    # end for

    end_bc_time = time()
    
#     u = alg.solve(K.tocsr(),f)
    u = alg.solve_sparse_tests(K.tocsr(),f)
#     u = alg.zeros(f.shape)
    
    end_solve_time = time()
    
    if stats:
        times['assembly'].append(  end_assmbly_time - start_time       )
        times['bc'].append(        end_bc_time      - end_assmbly_time )
        times['solve'].append(     end_solve_time   - end_bc_time      )
        times['total'].append(     end_solve_time   - start_time       )
#         print("Assembly time:  {0:.6f} s ".format( end_assmbly_time - start_time       ) )
#         print("Boundary time:  {0:.6f} s ".format( end_bc_time      - end_assmbly_time ) )
#         print("Solution time:  {0:.6f} s ".format( end_solve_time   - end_bc_time      ) )
        print("Total run time: {0:.6f} s ".format( end_solve_time   - start_time       ) )
        print()
    # end if 
    
    return u,K,f

#-----------------------------------------------------------------------------------------------------
    
def assemble_globals(Kel,fel,Kgl,fgl,elem):
    
    
    for i in range(elem.size):
        
        ii=int(elem[i])
        
        for j in range(elem.size):

            jj=int(elem[j])
        
            Kgl[ii,jj] = Kgl[ii,jj] + Kel[i,j]
            
        # end for 
        fgl[ii,0] = fgl[ii,0] + fel[i,0]
        
    # end for 

    
def analytic_solution(Th):
    return Th.x*(1-Th.x)*Th.y*(1-Th.y)

In [6]:
# dual, r, md2, om1n2, om1n10
fem.set_global_algebra(r)
alg = fem.get_global_algebra()

# Th = fem.square(1,1,he=0.003,quads=True,structured=True)
Th = fem.square(1,1,he=0.0015,quads=True,structured=True)
# Th = fem.square(1,1,he=0.1,quads=False,structured=True)
b = alg.number(0.5)
order = 1
# for i in range(1,100+1):
#     b += alg.e(i,order=order)
# # end for   
b = b**order
Th.x += b
Th.y += b

Th

< mesh (pyoti.real) object with 447561 nodes, 448900 elements of types ( point1 (4), line2 (2672), quad4 (446224) ) >

In [7]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []

for i in range(1):
# for i in range(3):
#     u,K,f = solve_2d_problem(Th)
    u,K,f = solve_2d_problem_prealloc(Th)
# print(u)

print('\n\nTimes:'+str(type(u)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )
print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )
# u_analytic = analytic_solution(Th)

Total run time: 28.456028 s 



Times:<class 'pyoti.real.dmat'>
- Avg Assembly time:  15.258637 s 
- Avg Boundary time:  0.002163 s 
- Avg Solution time:  13.195228 s 
- Avg Total run time: 28.456028 s 


In [8]:
K.tocsr().real

<447561x447561 sparse matrix of type '<class 'numpy.float64'>'
	with 4020025 stored elements in Compressed Sparse Row format>

In [ ]:
# 446000 Elements (Pre-alloc)
# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  14.151064 s 
# - Avg Boundary time:  0.002262 s 
# - Avg Solution time:  12.083795 s 
# - Avg Total run time: 26.237121 s 

# Std solver.
# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  22.552886 s 
# - Avg Boundary time:  0.003251 s 
# - Avg Solution time:  17.348347 s 
# - Avg Total run time: 39.904484 s

# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  23.590920 s 
# - Avg Boundary time:  0.002970 s 
# - Avg Solution time:  16.416110 s 
# - Avg Total run time: 40.010000 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  39.787570 s 
# - Avg Boundary time:  0.004179 s 
# - Avg Solution time:  75.329859 s 
# - Avg Total run time: 115.121608 s 




# Trunc_dot solver
# Times:<class 'pyoti.static.onumm1n1.omatm1n1'>
# - Avg Assembly time:  22.449261 s 
# - Avg Boundary time:  0.002927 s 
# - Avg Solution time:  13.408534 s 
# - Avg Total run time: 35.860722 s 
# Times:<class 'pyoti.static.onumm1n2.omatm1n2'>
# - Avg Assembly time:  23.479539 s 
# - Avg Boundary time:  0.002976 s 
# - Avg Solution time:  14.701312 s 
# - Avg Total run time: 38.183826 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  33.923870 s 
# - Avg Boundary time:  0.003041 s 
# - Avg Solution time:  44.063196 s 
# - Avg Total run time: 77.990107 s 

# Times:<class 'pyoti.sparse.matso'> (real only)
# - Avg Assembly time:  48.587235 s 
# - Avg Boundary time:  0.006627 s 
# - Avg Solution time:  13.885102 s 
# - Avg Total run time: 62.478965 s 

# Times:<class 'pyoti.sparse.matso'> (n=1,m=1, all nodal coordinates)
# - Avg Assembly time:  53.965517 s 
# - Avg Boundary time:  0.004058 s 
# - Avg Solution time:  17.867715 s 
# - Avg Total run time: 71.837290 s 

# Times:<class 'pyoti.sparse.matso'> (n=1,m=100, all nodal coordinates)
# Times:<class 'pyoti.sparse.matso'>
# - Avg Assembly time:  170.650918 s 
# - Avg Boundary time:  0.017645 s 
# - Avg Solution time:  120.130132 s 
# - Avg Total run time: 290.798695 s 

In [32]:
bout = b.copy()
%timeit alg.div(b,b,out=bout)

1.27 µs ± 14 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Kcsr = K.tocsr()
%timeit Kcsr.real.tocsc()

In [8]:
%timeit Kcsr.order

185 ms ± 1.58 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
Kcsc_r = Kcsr.real.tocsc()
lu = spla.splu(Kcsc_r)

In [1]:
K

NameError: name 'K' is not defined

In [14]:
%timeit lu = spla.splu(Kcsc_r)

11.1 s ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
dofs = f.shape[0]
rhs = np.zeros((dofs,20))
%timeit sol = lu.solve(rhs)

1.46 s ± 9.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
# Times:<class 'pyoti.real.dmat'>
# - Avg Assembly time:  4.063516 s 
# - Avg Boundary time:  0.001101 s 
# - Avg Solution time:  1.891620 s 
# - Avg Total run time: 5.956237 s 

# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  12.215592 s 
# - Avg Boundary time:  0.003618 s 
# - Avg Solution time:  22.003144 s 
# - Avg Total run time: 34.222354 s 



# Using TESTS
# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  11.699303 s 
# - Avg Boundary time:  0.004151 s 
# - Avg Solution time:  11.196514 s 
# - Avg Total run time: 22.899968 s 
#
# Changed the get/set im for the get order address form.
# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  11.407149 s 
# - Avg Boundary time:  0.001667 s 
# - Avg Solution time:  10.834901 s 
# - Avg Total run time: 22.243718 s 
#
# Using Out functions
# Times:<class 'pyoti.static.onumm1n10.omatm1n10'>
# - Avg Assembly time:  10.884407 s 
# - Avg Boundary time:  0.001730 s 
# - Avg Solution time:  10.750995 s 
# - Avg Total run time: 21.637132 s 







# USing tests
# Times:<class 'pyoti.static.mdnum2.mdmat2'>
# - Avg Assembly time:  7.153970 s 
# - Avg Boundary time:  0.001681 s 
# - Avg Solution time:  2.932703 s 
# - Avg Total run time: 10.088355 s 

In [7]:
u.real

array([[-2.97017026e-36],
       [-2.97017026e-36],
       [-2.97017026e-36],
       ...,
       [ 2.65709826e-05],
       [ 1.77675050e-05],
       [ 8.91051078e-06]])

In [12]:
uord = u.copy()
Kcsr=K.tocsr()
%timeit alg.dot(Kcsr,u)

128 ms ± 2.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%timeit alg.trunc_dot(1,Kcsr,1,u)

110 ms ± 1.22 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
Kcsr = K.tocsr()
Kcsr

<(112225, 112225) sparse matrix of OTI numbers with 
         1006009 stored elements in Compressed Sparse Row format>

In [42]:
Kord = Kcsr.zeros_like()
tmp = f.copy()
# %timeit alg.get_order_im(3,f)
%timeit alg.dot(Kcsr,f,out=tmp)

128 ms ± 309 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [72]:
times = {}
times['assembly'] = []
times['bc'] = []
times['solve'] = []
times['total'] = []

for i in range(3):
    u2,K2,f2 = solve_2d_problem_prealloc(Th)

print('\n\nTimes:'+str(type(u)))
print("- Avg Assembly time:  {0:.6f} s ".format(np.average(times['assembly'] ) ) )
print("- Avg Boundary time:  {0:.6f} s ".format(np.average(times['bc'])      ) )
print("- Avg Solution time:  {0:.6f} s ".format(np.average(times['solve'])   ) )
print("- Avg Total run time: {0:.6f} s ".format(np.average(times['total'])   ) )
# u_analytic = analytic_solution(Th)

Assembly time:  14.034225 s 
Boundary time:  0.002524 s 
Solution time:  3.459164 s 
Total run time: 17.495913 s 

Assembly time:  13.489687 s 
Boundary time:  0.002084 s 
Solution time:  3.181534 s 
Total run time: 16.673305 s 

Assembly time:  12.527748 s 
Boundary time:  0.002040 s 
Solution time:  3.218981 s 
Total run time: 15.748769 s 



Times:<class 'pyoti.sparse.matso'>
- Avg Assembly time:  13.350553 s 
- Avg Boundary time:  0.002216 s 
- Avg Solution time:  3.286560 s 
- Avg Total run time: 16.639329 s 


In [23]:
type(u)

pyoti.real.dmat

In [8]:
%timeit K.tocsr()

581 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:

%timeit K[10,10]

858 ns ± 39.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [7]:
type(K)

pyoti.sparse.lil_matso

In [ ]:
%

In [6]:
Th

< mesh (soti) object with 112225 nodes, 112896 elements of types ( point1 (4), line2 (1336), quad4 (111556) ) >

In [ ]:
# Mesh - 112896 (Quad4)
# Assembly time:  24.293029 s 
# Boundary time:  0.002895 s 
# Solution time:  4.505786 s 
# Total run time: 28.801710 s


# No Global Assembly
# Assembly time:  19.667007 s # Elemental computation
# Boundary time:  0.001495 s 
# Solution time:  0.001715 s 
# Total run time: 19.670217 s 

In [10]:
b = e(1)
%timeit K[10,10] = K[10,10] + b

903 ns ± 9.03 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [8]:
# import pyvista as pv
# p = pv.BackgroundPlotter()

grid = Th.to_pv(pd = [u.real],pd_names=['u'])
p.set_background('gray')
p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
# p.add_mesh(grid.warp_by_scalar(factor = 3), show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
p.show_bounds()
p.show_grid()

In [32]:
# import pyvista as pv
# p = pv.BackgroundPlotter()

grid = Th.to_pv(pd = [u_analytic.real],pd_names=['u analytic'])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
p.show_bounds()
p.show_grid()

In [12]:
# import pyvista as pv
# p = pv.BackgroundPlotter()

grid = Th.to_pv(pd = [np.abs(((u_analytic-u)).real)],pd_names=['Abs $\epsilon$ Error'])

p.clear()

# p.add_mesh(grid, show_edges=False, line_width=2,grid = True,categories=16,cmap='jet')
# p.add_mesh(grid, show_edges=True, line_width=2,grid = True,categories=256,cmap='jet')
p.add_mesh(grid.warp_by_scalar(factor = 3), show_edges=False, line_width=2,grid = True,categories=256,cmap='jet', scalar_bar_args={'interactive':True})
p.set_background('gray')

p.show_grid(xlabel='x', ylabel='y')

In [3]:
fem.hex8.allocate(intorder = 2)
fem.hex8.allocate_spatial(3,compute_Jinv = True)


In [8]:
a = oti.transpose(fem.hex8.N)
b = fem.hex8.N

%timeit oti.dot(a,b)

31 µs ± 325 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [7]:
oti.dot(a,b)

matsofe< nip: 8, 
(Integration point - 0) 
-------------------------
matso< shape: (8, 8), 
 - Column 0
(0,0) 0.240652
(1,0) 0.0644824
(2,0) 0.017278
(3,0) 0.0644824
(4,0) 0.0644824
(5,0) 0.017278
(6,0) 0.00462963
(7,0) 0.017278
 - Column 1
(0,1) 0.0644824
(1,1) 0.017278
(2,1) 0.00462963
(3,1) 0.017278
(4,1) 0.017278
(5,1) 0.00462963
(6,1) 0.00124051
(7,1) 0.00462963
 - Column 2
(0,2) 0.017278
(1,2) 0.00462963
(2,2) 0.00124051
(3,2) 0.00462963
(4,2) 0.00462963
(5,2) 0.00124051
(6,2) 0.000332392
(7,2) 0.00124051
 - Column 3
(0,3) 0.0644824
(1,3) 0.017278
(2,3) 0.00462963
(3,3) 0.017278
(4,3) 0.017278
(5,3) 0.00462963
(6,3) 0.00124051
(7,3) 0.00462963
 - Column 4
(0,4) 0.0644824
(1,4) 0.017278
(2,4) 0.00462963
(3,4) 0.017278
(4,4) 0.017278
(5,4) 0.00462963
(6,4) 0.00124051
(7,4) 0.00462963
 - Column 5
(0,5) 0.017278
(1,5) 0.00462963
(2,5) 0.00124051
(3,5) 0.00462963
(4,5) 0.00462963
(5,5) 0.00124051
(6,5) 0.000332392
(7,5) 0.00124051
 - Column 6
(0,6) 0.00462963
(1,6) 0.00124051
(2,6) 0.

In [10]:
fem.hex8.nip

8

In [25]:
x1 = np.zeros((8,8,1))
x2 = np.zeros((8,1,8))

In [26]:
def dot_fem(a,b):
    
    nip  = a.shape[0]
    nrow = a.shape[1]
    ncol = b.shape[2]
    
    res  = np.zeros((nip,nrow,ncol))
    
    for i in range(nip):
        res[i] = np.dot(a[i],b[i])
    # end for 
    
    return res

# end 

In [27]:
%timeit dot_fem(x1,x2)

22.3 µs ± 520 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
%timeit dot_fem(x2,x1)

18.4 µs ± 91.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [30]:
%timeit oti.dot_product(a,b)

3.23 µs ± 30.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [31]:
a.nip

8

In [2]:
import sympy as sym

In [3]:
x,y = sym.symbols("x y")
f = x*(1-x)*y*(1-y)

In [5]:
laplace = f.diff(x,2)+f.diff(y,2)

In [6]:
laplace.expand()

2*x**2 - 2*x + 2*y**2 - 2*y